In [1]:
# Already in that branch on server 

# !git clone -b dataloader https://github.com/airpods69/UnsupervisedVAD

Cloning into 'UnsupervisedVAD'...
remote: Enumerating objects: 3835, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 3835 (delta 53), reused 160 (delta 52), pack-reused 3666
Receiving objects: 100% (3835/3835), 119.14 MiB | 38.27 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

print(IN_COLAB)

False


##### Important Changes

in Segmenter/frame_extractor.py

Change path to `"./UnsupervisedVAD/Dataset/"` (Line 81) 

`class_name,video_name = write_path.split('/')[4], write_path.split('/')[5]` (Line 8)

In [2]:
frame_extractor = '''import cv2
import os

def write_path_to_annotation(write_path, frameNo, file):

    class_name,video_name = write_path.split('/')[4], write_path.split('/')[5]
    file.write("{}/{} 1 {} 0 \\n".format(class_name, video_name, frameNo))


def frame_extractor(path, video, frames_path):
    """
    Extracts Frame from path to video
    Args:
        Input: Path to video

        Output: 16 Frames stored into Folder(Foldername: Dataset/Path/)
    """

    class_name = path.split('/')[-1]

    frames_dir = os.path.join(frames_path, f'Frames/{class_name}/{video}')

    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)

    print(f"Path to Video is: {os.path.join(path, video)}")

    capture = cv2.VideoCapture(os.path.join(path, video))

    video_dir = os.path.join(path, video)

    if not os.path.exists(video_dir):
        os.makedirs(video_dir)

    # Keeping track of Frames
    frameNo = 0
    frameCount = 0

    file = open('./UnsupervisedVAD/train.txt', 'a')

    while(True):

        success, frame = capture.read() # Success -> if frame read successfully or not

        frameNo += 1
        frameCount += 1
        if success:
            
            write_path = os.path.join(frames_dir ,'frame_{:05d}.jpg'.format(frameNo))
            print(f"Wrote {write_path}")
            
            cv2.imwrite(write_path, frame)
        else:
            break


    print(f'Number of frames in {video} is {frameCount}')
    write_path_to_annotation(write_path = str(write_path), frameNo=frameCount, file=file)
    capture.release()


def video_selector(path):
    """
    Selects the video and sends it for frame extraction
    Input: Path
    Output: None
    """

    path_dataset = os.path.join(path, 'Datasets') # Change according to location


    for i in os.listdir(path_dataset):
        for j in os.listdir(os.path.join(path_dataset, i)):
            class_dir = os.path.join(path_dataset, i) # Class directory
            print(os.path.join(class_dir, j))
            frame_extractor(class_dir, j, path)



path = "./UnsupervisedVAD/Dataset/"


# frame_extractor(path, video)
video_selector(path)
# print(os.system('pwd'))

print("Done")
'''

In [3]:
with open('/content/UnsupervisedVAD/Segmenter/frame_extractor.py', 'w') as f:
    f.write(frame_extractor)
    f.close()

## Actual Code

In [4]:
!rm -rf /content/UnsupervisedVAD/Dataset/Frames

In [5]:
!python /content/UnsupervisedVAD/Segmenter/frame_extractor.py

./UnsupervisedVAD/Dataset/Datasets/Abuse/Abuse009_x264.mp4
Path to Video is: ./UnsupervisedVAD/Dataset/Datasets/Abuse/Abuse009_x264.mp4
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00001.jpg
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00002.jpg
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00003.jpg
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00004.jpg
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00005.jpg
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00006.jpg
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00007.jpg
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00008.jpg
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00009.jpg
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00010.jpg
Wrote ./UnsupervisedVAD/Dataset/Frames/Abuse/Abuse009_x264.mp4/frame_00011

In [6]:
import numpy as np

In [7]:
from UnsupervisedVAD.video_dataset import VideoFrameDataset, ImglistToTensor

In [8]:
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
path = './UnsupervisedVAD/Dataset/Frames/'

transfrom = transforms.Compose([
            ImglistToTensor(),
            transforms.CenterCrop((256,256)),
            transforms.Resize((224, 224)),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
            ])

dataset = VideoFrameDataset(path, './UnsupervisedVAD/train.txt', num_segments=10, frames_per_segment=16, imagefile_template='frame_{:05d}.jpg', transform=None, test_mode=False)

In [9]:
sample = dataset[0]  # take first sample of dataset 
frames = sample[0] # Gets frames of sample

In [10]:
segments = [[]] * len(dataset) # Get empty list of empty lists for each video in dataset
for j in range(len(dataset)):

    sample = dataset[j]
    frames = sample[0]

    segment = []

    for i in range(len(frames)//16):
        segment_i = frames[16 * i: 16 * (i + 1)]
        segment.append(segment_i)


    segments[j] = segment

segments = np.array(segments)
segments.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  app.launch_new_instance()


(2, 10, 16)

In [11]:
vid=segments[0]
print(len(vid[1]))

16


In [12]:
for i in range(len(segments[0][1])):
    print(segments[0][1][i])

<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6BDB10>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6BDB90>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6BDC10>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6BDC90>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6BDD10>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6BDD90>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6BDE10>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6BD8D0>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6BDF10>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6ACCD0>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6BDF90>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6C10D0>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6C1150>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6C11D0>
<PIL.Image.Image image mode=RGB size=320x240 at 0x7F35FA6C1250>
<PIL.Image.Image image mode=RGB size=320

In [13]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

frameSize = (224, 224)
vid_tensors=[[]] * len(dataset)

#vid_tensors[j]= list of 10 tensors for the 10 segments of video no. j

for i in range(len(segments)): #get_video
    lt=[]
    #print('video no.',i)
    vid=segments[i]
    frameset=np.array(vid) #array of 10 segments
    #print('no. of segments in video ',i,': ',len(frameset))
    for seg_index in range(0,len(frameset)):
        #print('seg_index',seg_index)
        seg=frameset[seg_index]
        l=[]
        for k in range(0,len(seg)): #16 frames per segment
            #print('frame no.',k)
            pil_img=seg[k]
            cv_img=np.array(pil_img)
            cv_img=cv2.resize(cv_img,(112,112))
            #cv2_imshow(cv_img)
            #print(cv_img.shape)
            l.append(cv_img)
       
        t=tuple(l)
        
        x = np.stack(t, axis = -1)
        y= np.transpose(x, (3,2,1,0)) #tensor for one segment
        #print('tensor for segment no. ', seg_index)
        #print('appending tensor for this segment')
        #print('vdi',vid_tensors[i])
        lt.append(y)
        #print('current shape',np.array(vid_tensors[0]).shape,np.array(vid_tensors[1]).shape)
    vid_tensors[i]=lt
    
    #print(i,len(vid_tensors[i]))

#cv2.destroyAllWindows()
vid_tensors=np.array(vid_tensors)
print(vid_tensors.shape)




(2, 10, 16, 3, 112, 112)


In [14]:
a=np.array(vid_tensors[0]).shape
print(a)

(10, 16, 3, 112, 112)


In [15]:
import os
import json

video_path = '/content/outpy.avi'
output_file = '/content/outpy.npy'
vids=vid_tensors.shape[0]
segs=vid_tensors.shape[1]
count=0
ls=[]
for v in range(0,vids):
    for s in range(0,segs):
        count+=1
        tens=vid_tensors[v][s]
        op_d={'video': tens, 'input': video_path, 'output': output_file}
        op_d_new=op_d
        op_d_new['video']=np.array(op_d['video']).tolist()
        ls.append(op_d_new)
        print('writing line ',count)

json_object=json.dumps(ls)
with open("sample.json", "w") as outfile:
    outfile.write(json_object)
    


writing line  1
writing line  2
writing line  3
writing line  4
writing line  5
writing line  6
writing line  7
writing line  8
writing line  9
writing line  10
writing line  11
writing line  12
writing line  13
writing line  14
writing line  15
writing line  16
writing line  17
writing line  18
writing line  19
writing line  20


In [16]:
import torch
torch.cuda.is_available()
feats=[[]]*(segs*vids)
print(np.array(feats).shape)

(20, 0)


In [17]:
!git clone https://github.com/antoine77340/video_feature_extractor

Cloning into 'video_feature_extractor'...
remote: Enumerating objects: 101, done.
remote: Total 101 (delta 0), reused 0 (delta 0), pack-reused 101
Receiving objects: 100% (101/101), 163.69 KiB | 2.24 MiB/s, done.
Resolving deltas: 100% (44/44), done.


## Edit

In [18]:
extract = """
import torch as th
import math
import numpy as np
from video_loader import VideoLoader
from torch.utils.data import DataLoader
import argparse
from model import get_model
from preprocessing import Preprocessing
from random_sequence_shuffler import RandomSequenceSampler
import torch.nn.functional as F

parser = argparse.ArgumentParser(description='Easy video feature extractor')

parser.add_argument(
    '--jsn',
    type=str,
    help='input json with video info')
parser.add_argument('--batch_size', type=int, default=64,
                            help='batch size')
parser.add_argument('--type', type=str, default='2d',
                            help='CNN type')
parser.add_argument('--half_precision', type=int, default=1,
                            help='output half precision float')
parser.add_argument('--num_decoding_thread', type=int, default=4,
                            help='Num parallel thread for video decoding')
parser.add_argument('--l2_normalize', type=int, default=1,
                            help='l2 normalize feature')
parser.add_argument('--resnext101_model_path', type=str, default='model/resnext101.pth',
                            help='Resnext model path')
args = parser.parse_args()

dataset = VideoLoader(
    args.jsn,
    framerate=1 if args.type == '2d' else 24,
    size=224 if args.type == '2d' else 112,
    centercrop=(args.type == '3d'),
)
n_dataset = len(dataset)
sampler = RandomSequenceSampler(n_dataset, 10)
loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=args.num_decoding_thread,
    sampler=sampler if n_dataset > 10 else None,
)
preprocess = Preprocessing(args.type)
model = get_model(args)
path='/content/output_features'
with th.no_grad():
    for k, data in enumerate(loader):
        input_file = data['input'][0]
        output_file = data['output'][0]
        if len(data['video'].shape) > 3:
            print('Computing features of video {}/{}: {}'.format(
                k + 1, n_dataset, input_file))
            video = data['video'].squeeze()
            if len(video.shape) == 4:
                video = preprocess(video)
                n_chunk = len(video)
                features = th.cuda.FloatTensor(n_chunk, 2048).fill_(0)
                n_iter = int(math.ceil(n_chunk / float(args.batch_size)))
                for i in range(n_iter):
                    min_ind = i * args.batch_size
                    max_ind = (i + 1) * args.batch_size
                    video_batch = video[min_ind:max_ind].cuda()
                    batch_features = model(video_batch)
                    if args.l2_normalize:
                        batch_features = F.normalize(batch_features, dim=1)
                    features[min_ind:max_ind] = batch_features
                features = features.cpu().numpy()
                if args.half_precision:
                    features = features.astype('float16')
                np.save(path+'/'+'output'+str(k+1)+'.npy', features)
        else:
            print('Video {} already processed.'.format(input_file))
"""

In [19]:
video_loader = '''import torch as th
from torch.utils.data import Dataset
import pandas as pd
import os
import numpy as np
import ffmpeg
import json


class VideoLoader(Dataset):
    """Pytorch video loader."""

    def __init__(
            self,
            jsn,
            framerate=1,
            size=112,
            centercrop=False,
    ):
        """
        Args:
        """
        with open(jsn,"r") as f:
            self.vid_list=json.load(f) #list of 20 dictionaries
        self.centercrop = centercrop
        self.size = size
        self.framerate = framerate

    def __len__(self):
        return len(self.vid_list)

    def _get_video_dim(self, vid_dict):

        vid_t=vid_dict['video'].shape
        height=vid_t[2]
        width=vid_t[3]
        self.size=(height,width)
        return height, width

    def _get_output_dim(self, h, w):
        if isinstance(self.size, tuple) and len(self.size) == 2:
            return self.size
        elif h >= w:
            return int(h * self.size / w), self.size
        else:
            return self.size, int(w * self.size / h)

    def __getitem__(self, idx):
        vid_dict=self.vid_list[idx]
        vid_dict['video']=th.from_numpy(np.array(vid_dict['video']))
        h,w=self._get_video_dim(vid_dict)
        (height, width) = self._get_output_dim(h, w)
        return vid_dict

'''

In [20]:
with open('/content/video_feature_extractor/extract.py', 'w') as f:
    f.write(extract)
    f.close()

In [21]:
with open('/content/video_feature_extractor/video_loader.py', 'w') as f:
    f.write(video_loader)
    f.close()

In [22]:
!wget https://www.rocq.inria.fr/cluster-willow/amiech/howto100m/models/resnext101.pth

--2022-09-04 11:35:09--  https://www.rocq.inria.fr/cluster-willow/amiech/howto100m/models/resnext101.pth
Resolving www.rocq.inria.fr (www.rocq.inria.fr)... 128.93.96.7
Connecting to www.rocq.inria.fr (www.rocq.inria.fr)|128.93.96.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 194020261 (185M)
Saving to: ‘resnext101.pth’

resnext101.pth      100%[===================>] 185.03M  9.65MB/s    in 20s     

2022-09-04 11:35:30 (9.20 MB/s) - ‘resnext101.pth’ saved [194020261/194020261]



In [23]:
!pip install ffmpeg
!pip install ffprobe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=bf7862631bfd4ce8a41cd92408e8e68a704fe91f7449f770bb88ad90f5efe3ef
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffprobe: filename=ffprobe-0.5-py3-none-any.whl size=3430 sha256=c0b641912b2126148eaef97cfd40b2f048e62a768d96bdac5965799cb52f1c8f
  Stored in directory: /root/.cache/pip/wheels/c8/68/e3/90be03cf7d5afcc4d97f533397c6a3522d0cc9a3288695fcc3
Successfully built ffprobe


In [27]:
import ffmpeg
import os

os.mkdir('/content/output_features/')

In [28]:
!python ./video_feature_extractor/extract.py --jsn="sample.json" --type=3d --batch_size=1 --resnext101_model_path=/content/resnext101.pth

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Loading 3D-ResneXt-101 ...
loaded
Computing features of video 1/20: /content/outpy.avi
Computing features of video 2/20: /content/outpy.avi
Computing features of video 3/20: /content/outpy.avi
Computing features of video 4/20: /content/outpy.avi
Computing features of video 5/20: /content/outpy.avi
Computing features of video 6/20: /content/outpy.avi
Computing features of video 7/20: /content/outpy.avi
Computing features of video 8/20: /content/outpy.avi
Computing features of video 9/20: /content/outpy.avi
Computing features of video 10/20

In [29]:
for k in range(1,(vids)*(segs)+1):
  z=np.load('/content/output_features/output'+str(k)+'.npy')
  z=np.array(z,dtype=np.float32)
  feats[k-1]=z


In [33]:
np.array(feats).shape

(20, 1, 2048)

In [31]:
# Making clean dataset for pre-training purpose.
clean_feat = []

for t in range(len(feats) - 1):
  diff = feats[t+1] - feats[t]
  value = np.linalg.norm(diff)
  if(value <= 0.7):
    clean_feat.append(feats[t+1])

clean_feat = np.array(clean_feat)

In [32]:
clean_feat.shape

(11, 1, 2048)

In [34]:
import torch
import torch.nn as nn

class AE(nn.Module):
    """
    AutoEncoder class implemented according to the paper
        FC[2048, 1024, 512, 256, 512, 1024, 2048]
    """

    def __init__(self):
        super().__init__()

        img_size = 2048

        self.encoder = nn.Sequential(
                # nn.Linear(img_size, 2048),
                # nn.ReLU(),
                #
                nn.Linear(2048, 1024),
                nn.ReLU(),

                nn.Linear(1024, 512),
                nn.ReLU(),

                nn.Linear(512, 256)
        )

        self.decoder = nn.Sequential(
                nn.Linear(256, 512),
                nn.LeakyReLU(0.3),

                nn.Linear(512, 1024),
                nn.LeakyReLU(0.3),

                nn.Linear(1024, 2048),
                # nn.LeakyReLU(0.3),

                # nn.Linear(2048, img_size),

                nn.Sigmoid()
        )

    def forward(self, img):
        encoded = self.encoder(img)
        decoded = self.decoder(encoded)

        return decoded

In [35]:
import torch
from torch.optim import RMSprop
from torch.utils.data import DataLoader

#from dataset import *

import matplotlib.pyplot as plt
from tqdm import tqdm

train_dataset=DataLoader(clean_feat,batch_size=1,shuffle=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model dec
netG = AE()

# reconstruction loss (MSE)
loss_function = torch.nn.MSELoss()

#RMSprop optimizer with a learning rate of 0.00002,
#momentum 0.60, for 15 epochs on training data with batch size 8192
optimizer = RMSprop(netG.parameters(), lr=0.00002, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.6, centered=False)


epochs = 15
outputs = []
losses = []
for epoch in range(epochs):

    running_loss = 0.
    print("Epoch No.: {}".format(epoch))

    #for (image, inputs) in tqdm(train_dataset):
    for i,j in tqdm(enumerate(train_dataset)):
        # Output of Autoencoder
        reconstructed = netG.forward(j)
        # Calculating the loss function
        loss = loss_function(reconstructed, j)
        # The gradients are set to zero,
        # the gradient is computed and stored.
        # .step() performs parameter update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Storing the losses in a list for plotting
        losses.append(loss)
        running_loss += loss.item() * j.size(0)

    epoch_loss = running_loss / len(train_dataset)
    # epoch_acc = running_corrects / len(normal_train_dataset) * 100.
    print("Loss: {}".format(epoch_loss))
    outputs.append((epochs, i, reconstructed))

# Defining the Plot Style
# plt.style.use('fivethirtyeight')
# plt.xlabel('Iterations')
# plt.ylabel('Loss')

print(type(losses))

# Plotting the last 100 values
# print(torch.tensor.detach().numpy().array(losses[-100:]).shape)
losses = [i.item() for i in losses]

# plt.plot(losses[-100:])
# plt.show()

losses = np.array(losses)

np.save("./losses.npy", losses)
path='model.pth'
torch.save(netG,path)


Epoch No.: 0


11it [00:00, 18.40it/s]


Loss: 0.21822435476563193
Epoch No.: 1


11it [00:00, 23.67it/s]


Loss: 0.03494255497406067
Epoch No.: 2


...

Loss: 0.00020110155111813748
Epoch No.: 13


11it [00:00, 22.39it/s]


Loss: 0.00019579332464755598
Epoch No.: 14


11it [00:00, 21.38it/s]

Loss: 0.0001915206164333292
<class 'list'>


In [36]:
#generate psuedo labels
#load model
G=torch.load(path)

In [37]:
print(G)

AE(
  (encoder): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.3)
    (2): Linear(in_features=512, out_features=1024, bias=True)
    (3): LeakyReLU(negative_slope=0.3)
    (4): Linear(in_features=1024, out_features=2048, bias=True)
    (5): Sigmoid()
  )
)


In [38]:
labels=[0]*(len(train_dataset))
losses=[]
with torch.no_grad():
    for i,j in tqdm(enumerate(train_dataset)):
      # Output of Autoencoder
      reconstructed = netG.forward(j)
      # Calculating the loss function
      #loss = loss_function(reconstructed, j)
      print(j.shape)
      j1=j.cpu().detach().numpy().reshape((2048,))
      r1=reconstructed.cpu().detach().numpy().reshape((2048,))
      x=[0]*(len(j1))
      for i in range(0,len(j1)):
        x[i]=j1[i]-r1[i]
      l=np.linalg.norm(x)
      losses.append(l)
losses=np.array(losses)
sr=np.std(losses)
ur=np.mean(losses)
th=ur+sr
for i in range(0,len(losses)):
  if(losses[i]>=th):
    labels[i]=1
labels = [np.array([i],dtype=np.float32) for i in labels]
labels

11it [00:00, 275.64it/s]

torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])


[array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([1.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32)]

In [39]:
import torch
import torch.nn as nn

class Discriminator(nn.Module):
    """
    Discriminator model made as per paper [https://arxiv.org/pdf/2203.03962.pdf]
    Fully Connected Layer
    FC[2048, 512, 32, 1]
    """

    def __init__(self) -> None:
        super().__init__()

        self.fc1 = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(512, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.LeakyReLU(0.3)
        self.sigmoid=nn.Sigmoid()

    def forward(self, inp):

        out = self.fc1(inp)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.sigmoid(out)

        return out

In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model dec
netD = Discriminator()

# reconstruction loss (MSE)
loss_function = torch.nn.BCELoss()

#RMSprop optimizer with a learning rate of 0.00002,
#momentum 0.60, for 15 epochs on training data with batch size 8192
optimizer = RMSprop(netD.parameters(), lr=0.00002, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.6, centered=False)


epochs = 15
outputs = []
losses = []
for epoch in range(epochs):

    running_loss = 0.
    print("Epoch No.: {}".format(epoch))

    #for (image, inputs) in tqdm(train_dataset):
    for i,j in tqdm(enumerate(train_dataset)):
    #for i,j in train_dataset:
        # Output of Autoencoder
        output = netD.forward(j).reshape(1,)
        #print(output)
        # Calculating the loss function
        #print(output.shape)
        #print(torch.tensor(labels[i]).shape)
        loss = loss_function(output,torch.tensor(labels[i]))
        # The gradients are set to zero,
        # the gradient is computed and stored.
        # .step() performs parameter update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Storing the losses in a list for plotting
        losses.append(loss)
        running_loss += loss.item() * j.size(0)

    epoch_loss = running_loss / len(train_dataset)
    # epoch_acc = running_corrects / len(normal_train_dataset) * 100.
    print("Loss: {}".format(epoch_loss))
    outputs.append((epochs, i, reconstructed))

# Defining the Plot Style
# plt.style.use('fivethirtyeight')
# plt.xlabel('Iterations')
# plt.ylabel('Loss')

print(type(losses))

# Plotting the last 100 values
# print(torch.tensor.detach().numpy().array(losses[-100:]).shape)
losses = [i.item() for i in losses]

# plt.plot(losses[-100:])
# plt.show()

losses = np.array(losses)

np.save("./losses_D.npy", losses)
path='model_D.pth'
torch.save(netD,path)

Epoch No.: 0


11it [00:00, 138.78it/s]


Loss: 0.6635074615478516
Epoch No.: 1


11it [00:00, 140.27it/s]


Loss: 0.5695210884917866
Epoch No.: 2


...

Loss: 0.33046326447616925
Epoch No.: 13


11it [00:00, 175.03it/s]


Loss: 0.3026228316805579
Epoch No.: 14


11it [00:00, 167.01it/s]

Loss: 0.3232834088531407
<class 'list'>


In [41]:
#pseudo labels from D
path='model_D.pth'
D=torch.load(path)
print(D)
labels_d=[0]*(len(train_dataset))
losses_d=[]
with torch.no_grad():
    for i,j in tqdm(enumerate(train_dataset)):
        output = netD.forward(j).reshape(1,)
        loss = loss_function(output,torch.tensor(labels[i]))
        losses_d.append(loss)
losses_d=np.array(losses_d)
sr=np.std(losses_d)
ur=np.mean(losses_d)
th=ur+(0.1)*sr
for i in range(0,len(losses_d)):
  if(losses_d[i]>=th):
    labels_d[i]=1
labels_d = [np.array([i],dtype=np.float32) for i in labels_d]
labels_d

Discriminator(
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (relu): LeakyReLU(negative_slope=0.3)
  (sigmoid): Sigmoid()
)


11it [00:00, 1645.06it/s]


[array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([1.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32),
 array([0.], dtype=float32)]

In [ ]:
# we know have pretrained G and D available

In [ ]:
#collaborative learning loop:
#generate pseudo labels from D
#use negative learning (all ones approach) from D pseudo labels in loss function of G
#generate pseudo-labels from G iteratively during training



In [43]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

netD = torch.load('/content/model_D.pth')
netG = torch.load('/content/model.pth')
print(netD)
print(netG)

Discriminator(
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (relu): LeakyReLU(negative_slope=0.3)
  (sigmoid): Sigmoid()
)
AE(
  (encoder): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.3)
    (2): Linear(in_features=512, out_features=1024, bias=True)
    (3): LeakyReLU(negative_slope=0.3)
    (4): Linear(in_features=1024, out_features=2048, bias=True)
    (5): Sigmoid()
  )
)


In [50]:

loss_function_D = torch.nn.BCELoss() # D
loss_function_G = torch.nn.MSELoss() # G

#RMSprop optimizer with a learning rate of 0.00002,
#momentum 0.60, for 15 epochs on training data with batch size 8192
optimizer = RMSprop(netD.parameters(), lr=0.00002, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.6, centered=False)


epochs = 15
outputs = []
losses_G_whole = [] # what is the loss for each epoch
losses_D_whole = [] # what is the loss for each epoch
for epoch in range(epochs):

    # labelsG = labels
    # labelsD = labels_d
    losses_G = [] # what is the loss for each sample, used for psuedo labels generation.
    losses_D = [] # what is the loss for each sample, used for psuedo labels generation.


    running_loss_G = 0.
    running_loss_D = 0
    print("Epoch No.: {}".format(epoch))

    # Training of the autoencoder
    for idx,data in tqdm(enumerate(train_dataset)):
        # Output of Autoencoder
        reconstructed = netG.forward(data)
        optimizer.zero_grad()
        loss = 0
        label = labels_d[idx]

        # Negative Learning
        if(label == 0):
            loss = loss_function_G(reconstructed, data)
            loss.backward()
            losses_G.append(loss.item())
        else:
            loss = loss_function_G(reconstructed, torch.ones(2048).reshape((1,1,-1)))
            loss.backward()
            losses_G.append(loss.item())

        optimizer.step()
        running_loss_G += loss.item() * data.size(0) # running_loss_G is the sum of losses for all the 20 feature vectors 
    losses_G_whole.append(running_loss_G)  


    # Pseudo labels generated from the autoencoder
    labels=[0]*(len(train_dataset)) # labels will be generated for each batch of data and not whole train_dataset.
    # losses=[]
    # with torch.no_grad():
    #   for i,j in tqdm(enumerate(train_dataset)):
    #   # Output of Autoencoder
    #     reconstructed = netG.forward(j)
    #   # Calculating the loss function
    #   #loss = loss_function(reconstructed, j)
    #     # print(j.shape)
    #     j1=j.cpu().detach().numpy().reshape((2048,))
    #     r1=reconstructed.cpu().detach().numpy().reshape((2048,))
    #     x=[0]*(len(j1))
    #     for i in range(0,len(j1)):
    #       x[i]=j1[i]-r1[i]
    #     l=np.linalg.norm(x)
    #     losses.append(l)
    losses_G=np.array(losses_G)
    sr=np.std(losses_G)
    ur=np.mean(losses_G)
    th=ur+sr
    for loss in range(0,len(losses_G)):
      if(losses_G[loss]>=th):
        labels[loss]=1
    labels = [np.array([label],dtype=np.float32) for label in labels]

    # Training of the discriminator
    for idx,data in tqdm(enumerate(train_dataset)):
        # output of discriminator
      output = netD.forward(data).reshape(1,)

      loss_D = loss_function_D(output,torch.tensor(labels[idx]))
        
      optimizer.zero_grad()
      loss_D.backward()
      optimizer.step()
        
      losses_D.append(loss_D.item())
      running_loss_D += loss_D.item() * data.size(0)
    losses_D_whole.append(running_loss_D)


    # Psuedo label generation from discriminator
    labels_d=[0]*(len(train_dataset))
    # losses_d=[]
    # with torch.no_grad():
    #   for i,j in tqdm(enumerate(train_dataset)):
    #     output = netD.forward(j).reshape(1,)
    #     loss = loss_function(output,torch.tensor(labels[i]))
    #     losses_d.append(loss)
    losses_D=np.array(losses_D)
    sr=np.std(losses_D)
    ur=np.mean(losses_D)
    th=ur+(0.1)*sr
    for loss in range(0,len(losses_D)):
      if(losses_D[loss]>=th):
        labels_d[loss]=1
    labels_d = [np.array([label],dtype=np.float32) for label in labels_d]
   

    epoch_loss = running_loss_D / len(train_dataset)
    # epoch_acc = running_corrects / len(normal_train_dataset) * 100.
    print("Loss: {}".format(epoch_loss))
    outputs.append((epochs, idx, reconstructed))


# np.save("./losses_G.npy", losses_G)
np.save("./losses_D.npy", losses_D_whole)


Epoch No.: 0


11it [00:00, 46.87it/s]
11it [00:00, 185.23it/s]


Loss: 0.3269770277494734
Epoch No.: 1


11it [00:00, 62.25it/s]
11it [00:00, 162.08it/s]


Loss: 0.3016436652026393
Epoch No.: 2


...

Loss: 0.3204576230861924
Epoch No.: 13


11it [00:00, 60.08it/s]
11it [00:00, 187.98it/s]


Loss: 0.3419015441428531
Epoch No.: 14


11it [00:00, 61.71it/s]
11it [00:00, 182.16it/s]

Loss: 0.3346078223125501


In [ ]:
# We will save the model using torch.state_dict() and load the pre-trained model afterwards
# in this co-operative training step, we train the generator and gather its losses for each sample(feature vector) 
# then we use those losses to generate pseudo labels from generator.
# P.S - We don't put generator on evaluation mode and generate the losses, we only take losses from the training part.
# then we use the pseudo labels from the generator to train the discriminator and take its losses to generate pseudo labels from discriminator.
# and the training loop goes on.